In [105]:
import pandas as pd

In [106]:
df = pd.read_csv('../dataset/train.csv')
df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [107]:
df.isnull().sum()

id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [108]:
df = df.drop(['id', 'CustomerId', 'Surname'], axis=1)

In [109]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

columns = ['Gender'] 

for col in df[columns]:
    enc = label_encoder.fit(df[col])
    encoded = label_encoder.transform(df[col])
    df[col] = encoded
    print(enc.classes_)
    print(enc.transform(enc.classes_))

['Female' 'Male']
[0 1]


In [110]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,668,France,1,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,627,France,1,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,678,France,1,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,581,France,1,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,716,Spain,1,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [111]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output = False).set_output(transform='pandas')

ohetransform = ohe.fit_transform(df[['Geography']])
ohetransform

df = pd.concat([df, ohetransform], axis=1).drop(columns=["Geography"])

In [112]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,668,1,33.0,3,0.00,2,1.0,0.0,181449.97,0,1.0,0.0,0.0
1,627,1,33.0,1,0.00,2,1.0,1.0,49503.50,0,1.0,0.0,0.0
2,678,1,40.0,10,0.00,2,1.0,0.0,184866.69,0,1.0,0.0,0.0
3,581,1,34.0,2,148882.54,1,1.0,1.0,84560.88,0,1.0,0.0,0.0
4,716,1,33.0,5,0.00,2,1.0,1.0,15068.83,0,0.0,0.0,1.0


In [113]:
X = df.drop('Exited', axis=1)
y = df['Exited']

In [114]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)

# Splitting the scaled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42)

# Displaying the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (132027, 12)
X_test shape: (33007, 12)
y_train shape: (132027,)
y_test shape: (33007,)


In [115]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 7, 10],
    'learning_rate': [0.01, 0.1, 0.5],
    'min_samples_split': [2],
    'min_samples_leaf': [1, 2, 4]
}

# Create the grid search model
grid_search = GridSearchCV(estimator=GradientBoostingRegressor(random_state=0),
                           param_grid=param_grid,
                           cv=2,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1,
                           verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters:", grid_search.best_params_)

# Get the best estimator
best_gbr = grid_search.best_estimator_

# Predicting on the testing set using the best model
y_pred_best = best_gbr.predict(X_test)

# Calculating the Mean Squared Error using the best model
mse_best = mean_squared_error(y_test, y_pred_best)
print("Best Mean Squared Error:", mse_best)

Fitting 2 folds for each of 81 candidates, totalling 162 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Best Mean Squared Error: 0.09768355099622045


In [116]:
df_test = pd.read_csv('../dataset/test.csv') 
df_test.head()  

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [117]:
id = df_test['id']
df_test = df_test.drop(['id', 'CustomerId', 'Surname'], axis=1)

In [118]:
columns = ['Gender'] 

for col in df_test[columns]:
    enc = label_encoder.fit(df_test[col])
    encoded = label_encoder.transform(df_test[col])
    df_test[col] = encoded
    print(enc.classes_)
    print(enc.transform(enc.classes_))

['Female' 'Male']
[0 1]


In [119]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output = False).set_output(transform='pandas')

ohetransform = ohe.fit_transform(df_test[['Geography']])
ohetransform

df_test = pd.concat([df_test, ohetransform], axis=1).drop(columns=["Geography"])

In [120]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_test_scaled = scaler.fit_transform(df_test)
df_test_scaled_df = pd.DataFrame(df_test_scaled, index=df_test.index, columns=df_test.columns)
df_test_scaled_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,-0.878176,-1.135026,-1.706504,-1.067887,-0.881274,0.820030,-1.746219,1.009580,0.967874,0.861202,-0.513237,-0.526975
1,0.329567,-1.135026,0.888990,-1.067887,-0.881274,-1.015806,0.572666,-0.990511,-0.790939,0.861202,-0.513237,-0.526975
2,-0.006609,-1.135026,-0.465181,0.713922,-0.881274,0.820030,0.572666,-0.990511,0.528413,0.861202,-0.513237,-0.526975
3,0.304665,0.881037,-0.239486,1.070284,-0.881274,-1.015806,0.572666,-0.990511,0.032150,0.861202,-0.513237,-0.526975
4,1.188684,0.881037,-0.013791,1.783008,1.050038,-1.015806,0.572666,-0.990511,0.539331,-1.161167,1.948417,-0.526975


In [121]:
test_result = rfr.predict(df_test_scaled_df)

In [122]:
df_result = pd.DataFrame({
    'id': id,
    'Exited': test_result,
})

In [123]:
df_result.to_csv('../result/submission_rfr2.csv', index=False)